In [ ]:
! pip install python-Levenshtein


import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
!pip install datasets


! pip install accelerate -U
! pip install transformers[torch]

In [ ]:
pip install --upgrade --force-reinstall pyarrow


In [ ]:
import pandas as pd
import os
from sklearn.utils import shuffle
import re
import numpy as np
from transformers import T5ForConditionalGeneration, AutoTokenizer
import nltk
import Levenshtein
from datasets import DatasetDict, Dataset
from transformers import DataCollatorWithPadding
from transformers import Trainer, TrainingArguments


In [ ]:
## install git  with bash command
#   !apt-get update
#   !apt-get install git

In [ ]:
## clone the folder of github

!git clone https://github.com/dessertlab/violent-python.git


Cloning into 'violent-python'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 28 (delta 4), reused 24 (delta 3), pack-reused 0
Receiving objects: 100% (28/28), 61.15 KiB | 3.40 MiB/s, done.
Resolving deltas: 100% (4/4), done.


## Merge the Datasets

In [ ]:
## paths of the folders

path_block = "/content/violent-python/block"
path_function = "/content/violent-python/function"
path_singleline = "/content/violent-python/singleline"


paths=[path_block, path_function, path_singleline]

In [ ]:
# Instantiate a dictionary


dict = {}
num = 0

# Iterate over the paths
for p in paths:
    num += 1
    i = 0
    # Iterate over the files in the folder
    for text in os.listdir(p):
        print(text)
        complete_path = os.path.join(p, text)
        with open(complete_path, "r") as file:
            content = file.readlines()  # Read all lines from the file

            # Apply cleaning to each line and store the cleaned lines in new_content
            new_content = [re.sub(r'\W+', ' ', line.strip()) for line in content]
            print(len(new_content))
            dict[f"{text}_{num}"] = new_content
        i += 1

python-test.in
17
python-dev.in
17
python-dev.out
17
python-train.out
137
python-test.out
17
python-train.in
137
python-test.in
7
python-dev.in
7
python-dev.out
7
python-train.out
58
python-test.out
7
python-train.in
58
python-test.in
113
python-dev.in
113
python-dev.out
113
python-train.out
902
python-test.out
113
python-train.in
902


In [ ]:
list(dict.keys())

['python-test.in_1',
 'python-dev.in_1',
 'python-dev.out_1',
 'python-train.out_1',
 'python-test.out_1',
 'python-train.in_1',
 'python-test.in_2',
 'python-dev.in_2',
 'python-dev.out_2',
 'python-train.out_2',
 'python-test.out_2',
 'python-train.in_2',
 'python-test.in_3',
 'python-dev.in_3',
 'python-dev.out_3',
 'python-train.out_3',
 'python-test.out_3',
 'python-train.in_3']

In [ ]:
names_train_in = [
    'python-train.in_1',
    'python-train.in_2',
    'python-train.in_3'
]

names_train_out = [
    'python-train.out_1',
    'python-train.out_2',
    'python-train.out_3'
]

names_validation_in = [
    'python-dev.in_1',
    'python-dev.in_2',
    'python-dev.in_3'
]

names_validation_out = [
    'python-dev.out_1',
    'python-dev.out_2',
    'python-dev.out_3'
]

names_test_in = [
    'python-test.in_1',
    'python-test.in_2',
    'python-test.in_3'
]

names_test_out = [
    'python-test.out_1',
    'python-test.out_2',
    'python-test.out_3'
]

In [ ]:
total_name = [names_train_in, names_train_out, names_validation_in, names_validation_out, names_test_in, names_test_out]

# Initialize the lists
list_train_in = []
list_train_out = []
list_validation_in = []
list_validation_out = []
list_test_in = []
list_test_out = []

# List containing all the lists
total_list = [list_train_in, list_train_out, list_validation_in, list_validation_out, list_test_in, list_test_out]

# Update the lists
for lista, names in zip(total_list, total_name):
    for name in names:
        lista.extend(dict[name])



In [ ]:
# Print the lengths of the lists
print([len(x) for x in total_list])

[1097, 1097, 137, 137, 137, 137]


In [ ]:
## now it is the time to shuffle data in ordee to improve robustness for the training when we will do batches groups

seed_value = 50

# Creation of dataframe and shuffling
df_train = pd.DataFrame({'Input': list_train_in, 'Output': list_train_out})
df_train = shuffle(df_train, random_state=seed_value)

df_validation = pd.DataFrame({'Input':list_validation_in , 'Output': list_validation_out})
df_validation = shuffle(df_validation, random_state=seed_value)

df_test = pd.DataFrame({'Input': list_test_in, 'Output': list_test_out})
df_test = shuffle(df_test, random_state=seed_value)


In [ ]:
print("Length of df_train:", len(df_train))
print("Length of df_validation:", len(df_validation))
print("Length of df_test:", len(df_test))

Length of df_train: 1097
Length of df_validation: 137
Length of df_test: 137


In [ ]:
## modol of our choice


##  T5 (Text-To-Text Transfer Transformer)

## class T5ForConditionalGeneration
## checkpoint =  "t5-small"

### class MarianMTModel
##  checkpoint =  "Helsinki-NLP/opus-mt-en-roa"


## Model,  zero-shot learning and Evaluation ("t5-small")

In [ ]:



# Load the tokenizer and the model

checkpoint = "t5-small"
device = "cuda" # for GPU usage or "cpu" for CPU usage
max_seq_length = 512
tokenizer = AutoTokenizer.from_pretrained(checkpoint, max_length=max_seq_length)
model = T5ForConditionalGeneration.from_pretrained(checkpoint).to(device)








/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
## for general purpose

input_texts = ["Write a Python function to add two numbers.",
              "Define a main function that opens a file.",
              "Declare a variable of type int."]


for sample in input_texts:
  # Tokenize the input text
  input_ids = tokenizer.encode(sample,  return_tensors="pt").to(device)

  print("\n****************\n")
  # Generate code based on the input text
  print("Intent: "+sample + "\n")
  output = model.generate(input_ids, max_length=64)

  # Decode the generated code
  generated_code = tokenizer.decode(output[0], skip_special_tokens=True)

  print("Output: \n")
  print(generated_code)




****************

Intent: Write a Python function to add two numbers.

Output: 

Schreiben Sie eine Python Funktion, um zwei Zahlen hinzufüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfüfü

****************

Intent: Define a main function that opens a file.

Output: 

Definifinit a main function that opens a file.

****************

Intent: Declare a variable of type int.

Output: 

Deklarez a variable de type int.


In [ ]:
## for our specific task





for i in np.arange(5):
    print("****************")
    print("Command:")
    print(df_test.iloc[i, 0])

    inputs = tokenizer.encode(df_test.iloc[i, 0], return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_length=60)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(result)

    print("\nGround Truth:")
    print(df_test.iloc[i, 1])





****************
Command:
Assign to soup the result of BeautufulSoup with argument html
Assign to soup the result of BeautufulSoup with argument html

Ground Truth:
soup BeautifulSoup html 
****************
Command:
For each cookie in the cookie_jar do nothing
Pour chaque cookie in the cookie_jar do nothing

Ground Truth:
for cookie in cookie_jar n pass
****************
Command:
Add the option u for url as type string with help equals to specify url address 
Add the option u for url as type string with help equals to specify url address.

Ground Truth:
parser add_option u dest url type string help specify url address 
****************
Command:
Assign to url the result of function str with argument row at position 0
Assign to url the result of function str with argument row at position 0

Ground Truth:
url str row 0 
****************
Command:
Assign to shellcode the tuple containing xbf x5c x2a x11 xb3 xd9 xe5 xd9 x74 x24 xf4 x5d x33 xc9 
shellcode the tuple containing xbf x5c x2a x11 x

 ### Evaluation


In [ ]:


# List to store predictions
predictions = []

# Perform predictions on the df_test dataframe
for i in df_test.iloc[:, 0]:
    # Encode input text
    inputs = tokenizer.encode(i, return_tensors="pt").to(device)
    # Generate output sequence
    outputs = model.generate(inputs, max_length=60)
    # Decode generated sequence
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions.append(result)

# Make sure df_test.iloc[:, 1] is a list of references
references = df_test.iloc[:, 1].tolist()



In [ ]:
def _levenshtein(str1, str2):
    return (1 - Levenshtein.distance(str1, str2) / max(len(str1), len(str2))) * 100

def mean_levenshtein_distance_percentage_similarity(list1, list2):
    similarities = []
    for s1, s2 in zip(list1, list2):
        similarities.append(_levenshtein(s1, s2))
    mean_similarity = np.mean(similarities)
    return similarities, mean_similarity




In [ ]:
similarities, mean_similarity = mean_levenshtein_distance_percentage_similarity(references, predictions)
print("Levenshtein similarity as percentage:", similarities)
print("Mean Levenshtein similarity as percentage:", mean_similarity)

Levenshtein similarity as percentage: [34.42622950819673, 53.191489361702125, 58.75, 18.055555555555557, 69.87951807228916, 25.71428571428571, 21.818181818181813, 37.2093023255814, 28.947368421052634, 21.875, 0.0, 13.124999999999998, 40.0, 15.789473684210531, 54.16666666666667, 43.28358208955224, 62.22222222222222, 18.181818181818176, 50.0, 20.779220779220775, 16.747572815533985, 40.36697247706422, 32.47863247863248, 30.573248407643316, 40.0, 2.7989821882951627, 49.056603773584904, 42.307692307692314, 82.08955223880598, 32.35294117647059, 33.333333333333336, 20.45454545454546, 62.19512195121951, 68.0, 11.111111111111116, 4.0000000000000036, 19.11764705882353, 35.29411764705882, 42.85714285714286, 26.74418604651163, 28.57142857142857, 18.181818181818176, 18.965517241379317, 23.376623376623375, 35.051546391752574, 31.25, 16.000000000000004, 88.52459016393442, 19.354838709677423, 39.34426229508197, 5.6521739130434785, 16.129032258064512, 30.708661417322837, 34.04255319148937, 16.049382716

## Transfer Learning, Fine-tuning  and Evaluation

In [ ]:
## transform the dataframe

dataset_train = Dataset.from_pandas(df_train)
dataset_validation = Dataset.from_pandas(df_validation)
dataset_test = Dataset.from_pandas(df_test)



In [ ]:
dataset_train

Dataset({
    features: ['Input', 'Output', '__index_level_0__'],
    num_rows: 1097
})

In [ ]:
def tokenize_function(example):
    input_encodings = tokenizer(example['Input'], truncation=True, padding=True)
    output_encodings = tokenizer(example['Output'], truncation=True, padding=True)

    # Creazione del nuovo oggetto Dataset con gli identificatori dei token e le maschere di attenzione
    tokenized_dataset = Dataset.from_dict({
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': output_encodings['input_ids'],  # Utilizziamo gli identificatori dei token di output come etichette
        'decoder_attention_mask': output_encodings['attention_mask']  # Utilizziamo gli identificatori dei token di output come input per il decoder
    })

    return tokenized_dataset

In [ ]:
tokenized_dataset_train = tokenize_function(dataset_train)


In [ ]:
tokenized_dataset_validation =  tokenize_function(dataset_validation)

In [ ]:
tokenized_dataset_train

Dataset({
    features: ['input_ids', 'attention_mask', 'target_ids', 'target_attention_mask'],
    num_rows: 1097
})

In [ ]:


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)





In [ ]:
## define the arguments

args = TrainingArguments(
    output_dir="/content/violent-python/model",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
)

In [ ]:
trainer = Trainer(
model=model,
tokenizer=tokenizer,
args=args,
data_collator=data_collator,
train_dataset=tokenized_dataset_train,
eval_dataset=tokenized_dataset_validation,
)

In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,0.287583
2,No log,0.219624
3,No log,0.191905
4,0.265600,0.175357
5,0.265600,0.166936
6,0.265600,0.160294
7,0.265600,0.155603
8,0.150700,0.152738
9,0.150700,0.151162
10,0.150700,0.150382


TrainOutput(global_step=1380, training_loss=0.18951616701872454, metrics={'train_runtime': 337.2618, 'train_samples_per_second': 32.527, 'train_steps_per_second': 4.092, 'total_flos': 475567828500480.0, 'train_loss': 0.18951616701872454, 'epoch': 10.0})

In [ ]:

#model.module.save_pretrained('save_here')

## Evaluation of the results

In [ ]:
# List to store predictions
predictions2 = []

# Perform predictions on the df_test dataframe
for i in df_test.iloc[:, 0]:
    # Encode input text
    inputs = tokenizer.encode(i, return_tensors="pt").to(device)
    # Generate output sequence
    outputs = model.generate(inputs, max_length=60)
    # Decode generated sequence
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions2.append(result)

# Make sure df_test.iloc[:, 1] is a list of references
references = df_test.iloc[:, 1].tolist()


In [ ]:
similarities, mean_similarity = mean_levenshtein_distance_percentage_similarity(references, predictions2)
print("Levenshtein similarity as percentage:", similarities)
print("Mean Levenshtein similarity as percentage:", mean_similarity)

Levenshtein similarity as percentage: [91.66666666666666, 32.25806451612904, 60.60606060606061, 92.85714285714286, 98.48484848484848, 50.0, 93.75, 61.111111111111114, 46.15384615384615, 73.33333333333334, 50.0, 62.16216216216216, 76.0, 0.0, 50.0, 74.28571428571429, 96.55172413793103, 80.0, 64.28571428571428, 42.85714285714286, 0.0, 60.3448275862069, 48.4375, 27.388535031847137, 96.42857142857143, 59.09090909090908, 58.82352941176471, 94.28571428571428, 96.49122807017544, 94.73684210526316, 36.0, 45.833333333333336, 56.707317073170735, 67.0, 11.111111111111116, 68.75, 97.5609756097561, 40.0, 96.7741935483871, 95.94594594594594, 75.92592592592592, 57.14285714285714, 12.0, 46.42857142857143, 97.14285714285714, 94.11764705882352, 39.13043478260869, 96.49122807017544, 69.23076923076923, 50.0, 58.82352941176471, 11.111111111111116, 48.484848484848484, 58.33333333333333, 63.1578947368421, 15.789473684210531, 51.61290322580645, 96.2962962962963, 85.41666666666666, 95.23809523809523, 93.3333333